In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.4 MB/s eta 0:00:00


In [ ]:
# facebook/bart-large-mnliで実行

In [26]:
#user_questionを形態素解析するためインストール
!pip install mecab-python3
!pip install unidic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.6/581.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7406 sha256=333cd4fde0cafb60986adc2d40e05a6673ff426c6ff43e5a8f60361e02203ddf
  Stored in directory: /root/.cache/pip/wheels/7a/72/72/1f3d654c345ea69d5d51b531c90daf7ba14cc555eaf2c64ab0
Successfully built unidic
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2


In [28]:
#辞書データをインストール
!python -m unidic download

download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:20<00:00, 25.1MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /usr/local/lib/python3.10/dist-packages/unidic/dicdir


In [30]:
from transformers import pipeline
import datetime
import MeCab
import unidic

In [14]:
#曜日取得用
def date_weekday():
    weekday = datetime.date.today().weekday()
    #月曜
    if weekday == 0:
      return "の営業時間は10時から20時です。(月曜)"
    #火曜
    elif weekday == 1:
      return "の営業時間は10時から20時です。(火曜)"
    #水曜
    elif weekday == 2:
      return "の営業時間は10時から20時です。(水曜)"
    #木曜
    elif weekday == 3:
      return "の営業時間は10時から20時です。(木曜)"
    #金曜
    elif weekday == 4:
      return "の営業時間は10時から20時です。(金曜)"
    #土曜
    elif weekday == 5:
      return "は休業日です。(土曜)"
    #日曜
    elif weekday == 6:
      return "は休業日です。(日曜)"
    else:
      return "不明です。"

In [41]:
def question_Analysis(user_question, specific_words):
    tagger = MeCab.Tagger()  # 「tagger = MeCab.Tagger('-d ' + unidic.DICDIR)」
    # MeCabを使用してテキストを解析
    result = tagger.parse(user_question)
    print(result)
    # 解析結果を分割し、単語ごとに処理
    for word_info in result.split("\n"):
        # 単語情報は「表層形\t品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音」の形式
        word_info_parts = word_info.split("\t")

        if len(word_info_parts) > 1:
            # 単語部分を取得
            word = word_info_parts[0]

            # 特定の単語が含まれている場合
            for specific_word in specific_words:
                if specific_word in word:
                    return True
    # 特定の単語が含まれていない場合
    return False

In [58]:
#ベースになる関数
def answer_question(question):
    # ゼロショット分類パイプラインを初期化
    classifier = pipeline("zero-shot-classification" )

    # 可能なカテゴリ
    candidate_categories = ["営業時間", "料金", "サービス内容", "予約方法","社長"]
    day_list = ["月曜日","火曜日","水曜日","木曜日","金曜日","土曜日","日曜日","今日"] #"今日"を追加


    # 分類を実行
    result = classifier(question, candidate_labels=candidate_categories, lang_id="ja")

    # 最も確率の高いカテゴリを取得
    best_category = result["labels"][0]

    # 各ラベルのスコアと分類結果を表示
    for label, score in zip(result["labels"], result["scores"]):
      print(f"ラベル: {label}, スコア: {score:.4f}")

    # カテゴリに応じた回答を生成（実際にはここでデータベースなどから情報を取得することも考えられます）
    if best_category == "営業時間":
        day_result = classifier(question, candidate_labels=day_list, lang_id="ja")
        day_category = day_result["labels"][0]
        print("")
        ## 曜日間におけるスコアリング → これだけでは"今日"ラベルがうまく機能しなかったため、形態素解析も組み込んだ。
        print("以下、day_resultのスコア")
        for label, score in zip(day_result["labels"], day_result["scores"]):
            print(f"ラベル: {label}, スコア: {score:.4f}")
        ## 以下、形態素解析
        specific_words = ["今日","本日"]
        if question_Analysis(user_question, specific_words):
            print("指定した単語が含まれています。")
            # 特定のラベルに0.5を加算
            specific_label = "今日"
            specific_index = day_result["labels"].index(specific_label)
            day_result["scores"][specific_index] += 0.5
            print(f"{specific_label} スコア: {day_result['scores'][specific_index]:.4f}")

            # ラベルとスコアを一時的なタプルのリストにまとめる
            label_score_pairs = [(label, score) for label, score in zip(day_result["labels"], day_result["scores"])]
            # スコアの大きい順にソート
            sorted_label_score_pairs = sorted(label_score_pairs, key=lambda x: x[1], reverse=True)
            # ソートされたリストからラベルとスコアを再度取り出す
            sorted_labels = [label for label, score in sorted_label_score_pairs]
            sorted_scores = [score for label, score in sorted_label_score_pairs]
            # day_result を更新
            day_result["labels"] = sorted_labels
            day_result["scores"] = sorted_scores

            day_category = day_result["labels"][0]
        else:
            print("指定した単語は含まれていません。")
        #--------------------------------
        if day_category == "月曜日":
            return "月曜日の営業時間は10時から20時です。"
        elif day_category == "火曜日":
            return "火曜日の営業時間は10時から20時です。"
        elif day_category == "水曜日":
            return "水曜日の営業時間は10時から20時です。"
        elif day_category == "木曜日":
            return "木曜日の営業時間は10時から20時です。"
        elif day_category == "金曜日":
            return "金曜日の営業時間は10時から20時です。"
        elif day_category == "今日":
            return "本日"+date_weekday()
        else:
            return "時間のことですか？\n          申し訳ありません、わかりません。"
    elif best_category == "料金":
        return "料金についての情報はAです。"
    elif best_category == "サービス内容":
        return "サービス内容についてはBです。"
    elif best_category == "予約方法":
        return "予約はCで行えます。"
    elif best_category == "社長":
        return "社長ブログをご覧ください。（http:// ...）"
    else:
        print("どの条件にもマッチしませんでした。")
        return "申し訳ありません、わかりません。"



---
以下、質問文


In [16]:
# ユーザーからの質問
user_question = "月曜日の営業時間は何時ですか？"
print("質問内容:",user_question)
print("")
# 質問に回答
response = answer_question(user_question)
print("")
print("回答内容:",response)
print("---------------------------------------------------------------------------")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


質問内容: 月曜日の営業時間は何時ですか？


ラベル: 営業時間, スコア: 0.8403
ラベル: 料金, スコア: 0.0672
ラベル: 社長, スコア: 0.0384
ラベル: 予約方法, スコア: 0.0276
ラベル: サービス内容, スコア: 0.0265


以下、day_resultのスコア
ラベル: 月曜日, スコア: 0.5123
ラベル: 木曜日, スコア: 0.2924
ラベル: 今日, スコア: 0.0549
ラベル: 日曜日, スコア: 0.0435
ラベル: 土曜日, スコア: 0.0411
ラベル: 金曜日, スコア: 0.0316
ラベル: 水曜日, スコア: 0.0183
ラベル: 火曜日, スコア: 0.0060


回答内容: 月曜日の営業時間は10時から20時です。
---------------------------------------------------------------------------


In [7]:
# ユーザーからの質問
user_question = "料金について教えて"
print("質問内容:",user_question)
print("")
# 質問に回答
response = answer_question(user_question)
print("")
print("回答内容:",response)
print("---------------------------------------------------------------------------")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


質問内容: 料金について教えて


ラベル: 料金, スコア: 0.8471
ラベル: 社長, スコア: 0.0504
ラベル: サービス内容, スコア: 0.0402
ラベル: 営業時間, スコア: 0.0321
ラベル: 予約方法, スコア: 0.0302


回答内容: 料金についての情報はAです。
---------------------------------------------------------------------------


In [ ]:
# ユーザーからの質問
user_question = "詳しく知りたい"
print("質問内容:",user_question)
print("")
# 質問に回答
response = answer_question(user_question)
print("")
print("回答内容:",response)
print("---------------------------------------------------------------------------")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


質問内容: 詳しく知りたい


ラベル: 社長, スコア: 0.3146
ラベル: 料金, スコア: 0.2429
ラベル: サービス内容, スコア: 0.2044
ラベル: 予約方法, スコア: 0.1261
ラベル: 営業時間, スコア: 0.1120


回答内容: 社長ブログをご覧ください。（http:// ...）
---------------------------------------------------------------------------


In [8]:
# ユーザーからの質問
user_question = "あいてる時間は何時ですか？"
print("質問内容:",user_question)
print("")
# 質問に回答
response = answer_question(user_question)
print("")
print("回答内容:",response)
print("---------------------------------------------------------------------------")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


質問内容: あいてる時間は何時ですか？


ラベル: 営業時間, スコア: 0.2372
ラベル: 社長, スコア: 0.2154
ラベル: 料金, スコア: 0.2091
ラベル: サービス内容, スコア: 0.2076
ラベル: 予約方法, スコア: 0.1306


以下、day_resultのスコア
ラベル: 土曜日, スコア: 0.1614
ラベル: 木曜日, スコア: 0.1537
ラベル: 月曜日, スコア: 0.1489
ラベル: 水曜日, スコア: 0.1480
ラベル: 日曜日, スコア: 0.1436
ラベル: 金曜日, スコア: 0.1328
ラベル: 火曜日, スコア: 0.1116


回答内容: 時間のことですか？
          申し訳ありません、わかりません。
---------------------------------------------------------------------------


In [11]:
# ユーザーからの質問
user_question = "サービス時間は何時ですか？"
print("質問内容:",user_question)
print("")
# 質問に回答
response = answer_question(user_question)
print("")
print("回答内容:",response)
print("---------------------------------------------------------------------------")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


質問内容: サービス時間は何時ですか？


ラベル: サービス内容, スコア: 0.2801
ラベル: 社長, スコア: 0.2227
ラベル: 営業時間, スコア: 0.1838
ラベル: 料金, スコア: 0.1677
ラベル: 予約方法, スコア: 0.1457


回答内容: サービス内容についてはBです。
---------------------------------------------------------------------------


In [10]:
# ユーザーからの質問
user_question = "時間は何時ですか？"
print("質問内容:",user_question)
print("")
# 質問に回答
response = answer_question(user_question)
print("")
print("回答内容:",response)
print("---------------------------------------------------------------------------")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


質問内容: 時間は何時ですか？


ラベル: 営業時間, スコア: 0.3834
ラベル: 料金, スコア: 0.1940
ラベル: 社長, スコア: 0.1859
ラベル: サービス内容, スコア: 0.1268
ラベル: 予約方法, スコア: 0.1099


以下、day_resultのスコア
ラベル: 土曜日, スコア: 0.1765
ラベル: 日曜日, スコア: 0.1504
ラベル: 木曜日, スコア: 0.1497
ラベル: 月曜日, スコア: 0.1453
ラベル: 水曜日, スコア: 0.1435
ラベル: 金曜日, スコア: 0.1337
ラベル: 火曜日, スコア: 0.1010


回答内容: 時間のことですか？
          申し訳ありません、わかりません。
---------------------------------------------------------------------------


In [9]:
# ユーザーからの質問
user_question = "時間は？"
print("質問内容:",user_question)
print("")
# 質問に回答
response = answer_question(user_question)
print("")
print("回答内容:",response)
print("---------------------------------------------------------------------------")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


質問内容: 時間は？


ラベル: 営業時間, スコア: 0.4461
ラベル: 社長, スコア: 0.2033
ラベル: 予約方法, スコア: 0.1320
ラベル: 料金, スコア: 0.1187
ラベル: サービス内容, スコア: 0.1000


以下、day_resultのスコア
ラベル: 土曜日, スコア: 0.1876
ラベル: 木曜日, スコア: 0.1546
ラベル: 月曜日, スコア: 0.1506
ラベル: 金曜日, スコア: 0.1393
ラベル: 日曜日, スコア: 0.1360
ラベル: 水曜日, スコア: 0.1326
ラベル: 火曜日, スコア: 0.0993


回答内容: 時間のことですか？
          申し訳ありません、わかりません。
---------------------------------------------------------------------------


In [ ]:
# ユーザーからの質問
user_question = "time？"
print("質問内容:",user_question)
print("")
# 質問に回答
response = answer_question(user_question)
print("")
print("回答内容:",response)
print("---------------------------------------------------------------------------")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


質問内容: time？


ラベル: 社長, スコア: 0.3180
ラベル: 料金, スコア: 0.2098
ラベル: サービス内容, スコア: 0.1769
ラベル: 予約方法, スコア: 0.1693
ラベル: 営業時間, スコア: 0.1259


回答内容: 社長ブログをご覧ください。（http:// ...）
---------------------------------------------------------------------------


In [ ]:
# ユーザーからの質問
user_question = "就業時間は？"
print("質問内容:",user_question)
print("\n")
# 質問に回答
response = answer_question(user_question)
print("\n")
print("回答内容:",response)
print("---------------------------------------------------------------------------")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


質問内容: 就業時間は？


ラベル: 営業時間, スコア: 0.3406
ラベル: 社長, スコア: 0.2297
ラベル: 料金, スコア: 0.1612
ラベル: 予約方法, スコア: 0.1481
ラベル: サービス内容, スコア: 0.1205


回答内容: 時間のことですか？
          申し訳ありません、わかりません。
---------------------------------------------------------------------------




---

以下、"今日"ラベル対応後

In [46]:
# ユーザーからの質問
user_question = "今日の対応時間は？"
print("質問内容:",user_question)
print("")
# 質問に回答
response = answer_question(user_question)
print("")
print("回答内容:",response)
print("---------------------------------------------------------------------------")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


質問内容: 今日の対応時間は？

ラベル: 営業時間, スコア: 0.4123
ラベル: 社長, スコア: 0.1908
ラベル: 料金, スコア: 0.1547
ラベル: 予約方法, スコア: 0.1350
ラベル: サービス内容, スコア: 0.1072

以下、day_resultのスコア
ラベル: 今日, スコア: 0.8631
ラベル: 土曜日, スコア: 0.0335
ラベル: 水曜日, スコア: 0.0235
ラベル: 日曜日, スコア: 0.0218
ラベル: 木曜日, スコア: 0.0182
ラベル: 金曜日, スコア: 0.0173
ラベル: 月曜日, スコア: 0.0172
ラベル: 火曜日, スコア: 0.0054
今日	名詞,普通名詞,副詞可能,,,,キョウ,今日,今日,キョー,今日,キョー,和,"","","","","","",体,キョウ,キョウ,キョウ,キョウ,"1","C3","",2509094191768064,9128
の	助詞,格助詞,,,,,ノ,の,の,ノ,の,ノ,和,"","","","","","",格助,ノ,ノ,ノ,ノ,"","名詞%F1","",7968444268028416,28989
対応	名詞,普通名詞,サ変可能,,,,タイオウ,対応,対応,タイオー,対応,タイオー,漢,"","","","","","",体,タイオウ,タイオウ,タイオウ,タイオウ,"0","C2","",5959086768267776,21679
時間	名詞,普通名詞,助数詞可能,,,,ジカン,時間,時間,ジカン,時間,ジカン,漢,"","","","","","B4WW7G9G,B4WW",体,ジカン,ジカン,ジカン,ジカン,"0","C2","",4884039274209792,17768
は	助詞,係助詞,,,,,ハ,は,は,ワ,は,ワ,和,"","","","","","",係助,ハ,ハ,ハ,ハ,"","動詞%F2@0,名詞%F1,形容詞%F2@-1","",8059703733133824,29321
？	補助記号,句点,,,,,,？,？,,？,,記号,"","","","","","",補助,,,,,"","","",15676664324608,57
EOS

指定した単語が含まれています。

回答内容: 本日の営業時間

In [59]:
# ユーザーからの質問
user_question = "本日の対応時間は？"
print("質問内容:",user_question)
print("")
# 質問に回答
response = answer_question(user_question)
print("")
print("回答内容:",response)
print("---------------------------------------------------------------------------")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


質問内容: 本日の対応時間は？

ラベル: 営業時間, スコア: 0.3452
ラベル: 社長, スコア: 0.2339
ラベル: 料金, スコア: 0.1459
ラベル: 予約方法, スコア: 0.1404
ラベル: サービス内容, スコア: 0.1345

以下、day_resultのスコア
ラベル: 木曜日, スコア: 0.3437
ラベル: 月曜日, スコア: 0.2928
ラベル: 今日, スコア: 0.1365
ラベル: 土曜日, スコア: 0.0901
ラベル: 日曜日, スコア: 0.0442
ラベル: 水曜日, スコア: 0.0398
ラベル: 金曜日, スコア: 0.0359
ラベル: 火曜日, スコア: 0.0169
本日	名詞,普通名詞,副詞可能,,,,ホンジツ,本日,本日,ホンジツ,本日,ホンジツ,漢,"","","","","","",体,ホンジツ,ホンジツ,ホンジツ,ホンジツ,"1","C1","",9597920500392448,34917
の	助詞,格助詞,,,,,ノ,の,の,ノ,の,ノ,和,"","","","","","",格助,ノ,ノ,ノ,ノ,"","名詞%F1","",7968444268028416,28989
対応	名詞,普通名詞,サ変可能,,,,タイオウ,対応,対応,タイオー,対応,タイオー,漢,"","","","","","",体,タイオウ,タイオウ,タイオウ,タイオウ,"0","C2","",5959086768267776,21679
時間	名詞,普通名詞,助数詞可能,,,,ジカン,時間,時間,ジカン,時間,ジカン,漢,"","","","","","B4WW7G9G,B4WW",体,ジカン,ジカン,ジカン,ジカン,"0","C2","",4884039274209792,17768
は	助詞,係助詞,,,,,ハ,は,は,ワ,は,ワ,和,"","","","","","",係助,ハ,ハ,ハ,ハ,"","動詞%F2@0,名詞%F1,形容詞%F2@-1","",8059703733133824,29321
？	補助記号,句点,,,,,,？,？,,？,,記号,"","","","","","",補助,,,,,"","","",15676664324608,57
EOS

指定した単語が含まれています。
今日 スコア